<a href="https://colab.research.google.com/github/abarraganc/Clases/blob/main/an%C3%A1lisis_de_Secuencias_Gen%C3%B3micas_Usando_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtener secuencias genómicas del NCBI y determinar características

En este comando lo primero que demos hacer es instalar biopython, tal cual como lo haciamos en nuestros computadores

In [ ]:
pip install biopython

Ahora vamos a importar la entrada al NCBI y a su lector de secuencias, e identificarnos con nuestro correo

In [ ]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = 'acbarragan@unicolmayor.edu.co'

Vamos a buscar una secuencia en específico, en este caso será el genóma de SARS-CoV-2 con ID: MN908947
Y la vemos esta entrada

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="MN908947", rettype="gb", retmode="text")
recs = list(SeqIO.parse(handle, 'gb'))
handle.close()
print(recs)

Vamos ahora a extraer la secuencia de DNA de esta entrada e imprimirla

In [ ]:
covid_dna = recs[0].seq
print(covid_dna)

Vamos a determinar la cantidad de nucleótidos que tiene esta secuencia

In [ ]:
print('El genoma de SARS-CoV-2 aislado en Wuhan contiene', len(covid_dna), 'nucleótidos')

Ahora determinaremos su peso molecular y su porcentaje de GC. Para ello debemos importar los comandos de cada uno e imprimir una respuesta.

In [ ]:
from Bio.SeqUtils import molecular_weight
print('El peso molecular del genoma de  SARS-CoV-2  es', molecular_weight(covid_dna), 'Da')
print('El peso molecular del genoma de  SARS-CoV-2  es',round(molecular_weight(covid_dna)), 'Da', ' \n')

from Bio.SeqUtils import gc_fraction
print('El porcentaje de GC del genoma de  SARS-CoV-2  es', (gc_fraction(covid_dna)*100), '%')
print('El porcentaje de GC del genoma de  SARS-CoV-2  es', round(gc_fraction(covid_dna)*100), '%')

Ahora vemos a contar la frecuencia de cada una de las bases nitrogenadas dentro del genoma

In [ ]:
count_nucleotides = {
    'A': covid_dna.count('A'),
    'T': covid_dna.count('T'),
    'C': covid_dna.count('C'),
    'G': covid_dna.count('G')
}
print(f'La frecuencia de nucleoótidos en el genoma es  {count_nucleotides}')

Haremos un gráfico de barras sencillo con esta frecuencia.
Para ello importaremos la librería de matplot y eligiremos cada una de las características de nuestro gráfico, como:
*   Ancho de barra: su ancho va aumentando desde 0.1 hacia arriba
*   Color de barra: podemos separar por , cada color que queremos para cada barra, puede escribirse el color en inglés, su inicial en inglés o el código hexadecimal de cada color
*   Nombre del eje X
*   Nombre del eje y
*   Título del gráfico

In [ ]:
import matplotlib.pyplot as plt
ancho_de_barra = 0.5
plt.bar(count_nucleotides.keys(), count_nucleotides.values(), ancho_de_barra, color=['#CEE7CC', 'r', 'orange', 'pink'])
plt.xlabel('Nucleótido')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de nucleótidos en el genoma de SARS-CoV-2')
plt.show()

# Transcribir y traducir un genoma

Como ya tenemos el genoma del SARS-CoV-2, es decir su secuencia de DNA, vamos a transcribirla para obtener el mRNA.
Para ello usamos la función transcribe que hará el cambio de T por U

In [ ]:
covid_mRNA=covid_dna.transcribe()
print(covid_mRNA)

Con este transcrito podemos entonces hacer su traducción a aminoácidos haciendo uso de la función translate.
De esta forma obtendremos la secuencia de aminoácidos en formato de 1 letra y el * se insertará cada vez que en el mRNA se encuentre un codón de parada

In [ ]:
covid_aa=covid_mRNA.translate()
print(covid_aa)

Vamos a contar la frecuencia con la que se encuentra cada uno de los aminoácidos.
Para esto vamos a importar la función counter.
*A diferencia de la función count que usamos previamente, con counter no necesitamos especificar uno a uno los aminoácidos para que los cuente, sino que python va a contar las veces que se repita cada caracter y nos dará una lista de caracteres y su frecuencia*

In [ ]:
from collections import Counter
frecuencia_aa = Counter(covid_aa)
print(frecuencia_aa)

#Ahora vamos a imprimir solo los 10 aminoácidos más frecuentes
print(' \n', frecuencia_aa.most_common(10))

#Al  usar esta función, cuenta como aminoácido el * entonces debemos eliminarlo y repetir la frecuencia de los 10 aminoácidos más comunes
del frecuencia_aa['*']
print(' \n', frecuencia_aa.most_common(10))

Crearemos ahora otro gráfica de frecuencias

In [ ]:
ancho_de_barra = 0.8
plt.bar(frecuencia_aa.keys(), frecuencia_aa.values(), ancho_de_barra, color=['#CEE7CC', '#DAE9F8', 'orange', 'pink', '#CCCCFF', '#009999'])
plt.xlabel('Aminoácido')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de aminoácidos codificados por el genoma de SARS-CoV-2')
plt.show()

Usando la función de suma podemos determinar el total de aminoácidos codificados en este genoma

In [ ]:
print('El genoma del Covid-19 codifica para', sum(frecuencia_aa.values()), 'aminoácidos')

Ahora vamos a determinar esos aminoácidos en cuantos polipéptidos se dividen, basado en cuantos codones de parada hay en este genoma. Para ello usaremos la función split que nos va a dividir la secuencia de aminoácidos en pequeñas secuencias, cada que haya un codón de parada. Y contaremos cuántas secuencias de polipéptidos se crean mediante la función len

In [ ]:
peptidos = covid_aa.split('*')
print(peptidos)
print( ' \n')
print('En el genoma de SARS-CoV-2 hay', len(peptidos), 'péptidos')


Como podemos ver hay secuencias peptídicas de diferentes tamaños, solo aquellas de más de 20 aa pueden considerarse como proteínas funcionales.
Para encontrar las potenciales proteínas vamos a crear una nueva lista con solo las secuencias de más de 20 aa, para ello usamos los operadores:
*   for: buscar dentro de cada iten de una lista
*   in: determinar dentro de qué lista va  abuscar
*   if: generar el condicional que se va a buscar en cada ítem de la lista
y contaremos cuántas proteìnas funcionales hay con la función len

In [ ]:
proteinas =[peptido for peptido in peptidos if len(peptido)>20]
print(proteinas)
print(len(proteinas))
print( ' \n')
print('De las', len(peptidos), 'secuencias peptídicas codificadas en el genoma de SARS-CoV-2, solo', len(proteinas), 'secuencias tienen más de 20 aminoácidos y pueden considerarse proteínas funcionales')

Organizaremos estas proteínas funcionales en orden ascentente, de menor a mayor longitud, usando la función sorted.
E imprimiremos la secuencia más corta y la más larga.
*Es importante recordar que python cuenta los elementos de una lista desde 0, por tanto, la proteína más corta será la 0 y la más larga la 72*

In [ ]:
tamaño_proteinas = sorted(proteinas, key = len)
print('La proteína más larga codificada en este genoma tiene', len(tamaño_proteinas[72]), 'aminoácidos y su secuencia es', tamaño_proteinas[72])
print('La proteína más corta codificada en este genoma tiene', len(tamaño_proteinas[0]), 'aminoácidosy su secuencia es', tamaño_proteinas[0])

Guardaremos la secuencia de la proteína más larga como un archivo fasta llamado protein_seq.fasta

In [ ]:
with open("protein_seq.fasta", "w") as file:
    file.write(f">covid protein\n{tamaño_proteinas[72]}")

Vamos a asegurarnos que python pueda leer nuestro archivo, impimiendolo por comando

In [ ]:
protein_seq = SeqIO.read("protein_seq.fasta", "fasta")
print(protein_seq.seq)

# Alineamiento a partir de una proteína del genoma

Ahora realizamos un alineamiento de nuestra secuencia con la base de datos del pdb (protein data bank) para determinar qué proteína es.
Solo pediremos los 5 mejores resultados

In [ ]:
from Bio.Blast import NCBIWWW
alineamiento = NCBIWWW.qblast("blastp", "pdb", protein_seq.seq)

from Bio import SearchIO
resultado_blast = SearchIO.read(alineamiento, 'blast-xml')
top_cinco_resultados= resultado_blast[0:5]

Imprimiremos los 5 mejores alineamientos usando los operadores for e in y separando cara una de las características de interés para nuestra búsqueda como ID de la secuencia, descripción, valor E, score normalizado bit y el alineamiento

In [ ]:
for resultado_blast in top_cinco_resultados:
    print(f"ID: {resultado_blast.id}")
    print(f"Descripción: {resultado_blast.description}")
    print(f"Valor E : {resultado_blast[0].evalue}")
    print(f"Score normalizado Bit:  {resultado_blast[0].bitscore}")
    print(f"Alineamiento:\n{resultado_blast[0].aln}")
    print()

# Visualización de una proteína

Antes de iniciar debemos instalar nglviw para poder visualizar nuestra proteína y habilitar su uso en google colab

In [ ]:
!pip install ipywidgets==7.7.2 nglview

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Dado que el pdb (protein data bank) es una base de datos que contiene múltiple información sobre proteínas, también almacena información estrucural que podemos descargar.
Para ello primero debemos obtener solo el número de ID de la secuencia

In [ ]:
seq_id = "pdb|6YYT|A"
id= seq_id.split("|")
print(id)
print(id[1])

Una vez obtenido este ID, usando el comando wget podemos recuperar los archivos de descripción de la estructura 3D de la secuencia

In [ ]:
!wget https://files.rcsb.org/download/6YYT.pdb

Recuperaremos esa información 3D desde nuestro archivo  y la vamos a llamar "estructura"

In [ ]:
from Bio.PDB import PDBParser
analizador = PDBParser()
estructura = analizador.get_structure('6YYT', '6YYT.pdb')
print(estructura)

Ya que tenemos la info estructural, podemos determinar por cuantas cadenas polipeptídicas está compuesta nuestra proteína y qué nombre recibe cada una.

In [ ]:
for cadena in estructura[0]:
    print(f'ID cadena: {cadena.id}')

In [ ]:
import nglview as nv
nv.show_biopython(estructura, gui=True)

In [ ]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
structure = PDBParser().get_structure('6YYT', '6YYT.pdb')
counter=0
ppb=PPBuilder()
for pp in ppb.build_peptides(structure):
     seq = pp.get_sequence()
     counter +=1
     print(f"Secuencia: {counter}, Longitud: {len(seq)}aa")
     print(seq)


In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
analyzed_seq = ProteinAnalysis(str(seq))
second_struc = analyzed_seq.secondary_structure_fraction()

for second in range(len(second_struc)):
  porcentaje= round(second_struc[second],2)*100
  estruc= ['alfa-hélice', 'loop', 'hoja-beta']
  print(f"De esta secuencia el procentaje {estruc[second]} es {porcentaje} %")


Support for third party widgets will remain active for the duration of the session. To disable support: